# Setup

In [307]:
import pandas as pd

In [308]:
packageData = pd.read_csv("package_data.csv")
oeeData = pd.read_csv("oee_data.csv")
errorMessages = pd.read_csv("error_messages_accumulated.csv")

# 1)

In [309]:
packageData["timestamp"] = pd.to_datetime(packageData["timestamp"])
oeeData["timestamp"] = pd.to_datetime(oeeData["timestamp"])

In [310]:
packageData["totalPacks"] = packageData["good_packs"] + packageData["reject_packs"]
totalPacks = packageData["totalPacks"].sum()

In [311]:
packageData["cyclesPerMinute"] = 60 / (packageData["timestamp"].diff() / pd.Timedelta(minutes=1))
oeeData["actual_cycles_per_minute"] = oeeData["actual_cycles_per_minute"].fillna(packageData["cyclesPerMinute"])
averageCyclesPerMinute = oeeData["actual_cycles_per_minute"].mean()
targetCyclesPerMinute = 10
cyclesPercent = (averageCyclesPerMinute / targetCyclesPerMinute) * 100

In [318]:
print("Total number of packs:", totalPacks)
print("Actual cycle power per minute: ", int(averageCyclesPerMinute))
print("Achieved cycle performance compared to planned cycle performance: ", int(cyclesPercent), "%")

Total number of packs: 6479336
Actual cycle power per minute:  6
Achieved cycle performance compared to planned cycle performance:  61 %


# 2)

In [313]:
headOfDowntime = errorMessages["code"].value_counts().head(3).reset_index()
headOfDowntime.columns = ["code", "number of errors"]

In [314]:
packageData["totalRejectPacks"] = packageData["reject_packs"].sum()
maxRejectPacksTimestamp = packageData.loc[packageData["totalRejectPacks"].idxmax(), "timestamp"]

In [315]:
print("Total number of bad packages:", packageData["reject_packs"].sum())
print("Most frequent downtime reasons for bad packaging:")
print(headOfDowntime)
print("Time with the most bad packs:", maxRejectPacksTimestamp)

Total number of bad packages: 293871
Most frequent downtime reasons for bad packaging:
   code  number of errors
0  1019               217
1  1010               155
2  2030               132
Time with the most bad packs: 2022-05-04 13:05:49.295000+00:00


# 3)

In [316]:
weeklyPacks = packageData.resample("W", on="timestamp").sum()["good_packs"]
averageWeeklyPacks = weeklyPacks.mean()
nextWeeksProjections = averageWeeklyPacks * 4

C:\Users\Maximilian\AppData\Local\Temp\ipykernel_13584\462802761.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weeklyPacks = packageData.resample("W", on="timestamp").sum()["good_packs"]


In [317]:
print("Average number of packs per week:", int(averageWeeklyPacks))
print("Forecast pack production for the next 4 weeks:", int(nextWeeksProjections))

Average number of packs per week: 475805
Forecast pack production for the next 4 weeks: 1903220
